In [1]:
from scripts.utils import utils
from scripts.model import OR_model

import pandas as pd
import itertools
import numpy as np
import matplotlib.pyplot as plt
import time
import gurobipy as gp
from gurobipy import GRB

In [2]:
def readData(pre=True):
    print("\n\n", "-"*10, "Reading data", "-"*10)
    df = pd.read_csv('dataset.csv')

    print("Data size:", df.shape)

    print("\n\n", "-"*10, "Do precalculations", "-"*10)
    start = time.time()

    utils_ = utils(df, pre=pre)

    print(f"Finish, uses {time.time() - start:.5f} seconds")

    return df, utils_

In [3]:
def getAllowedMoves(df, utils):
    B = df['Block'].unique()
    P_set = set()

    for b1, b2 in itertools.product(B, repeat=2):
        o1 = utils.getRoom(b1)
        o2 = utils.getRoom(b2)
        if abs(o1 - o2) <= 10:
            P_set.add((b1,b2))

    return list(P_set)

In [4]:
df, utils = readData(pre=False)
P = getAllowedMoves(df, utils)

# Hyperparameters
F = ['1']
l_v = [0,0,0,0,0]
u_v = [150,150,150,150,150]



 ---------- Reading data ----------
Data size: (1456, 7)


 ---------- Do precalculations ----------
Finish, uses 0.00000 seconds


In [5]:
model = OR_model(df=df, P=P, F=F, l_v=l_v, u_v=u_v, alpha=0.5, utils=utils)
model.init()
model.optimize()



 ---------- Creating model ----------
Set parameter Username
Set parameter LicenseID to value 2631632
Academic license - for non-commercial use only - expires 2026-03-05
New model created successfully.


 ---------- Reading sets from df and precompute ----------
Finish, uses 0.00300 seconds


 ---------- Setting decision variables ----------
Finish, uses 0.19000 seconds


 ---------- Setting constraints ----------
Setting additional constraints
Finish, uses 2.11963 seconds
Setting constraints (2)
Finish, uses 2.10498 seconds
Setting constraints (3)
Finish, uses 46.40154 seconds
Setting constraints (4)
Finish, uses 66.21167 seconds
Setting constraints (5)
Finish, uses 142.66099 seconds
---------- Setting objection ----------
Finish, uses 0.00200 seconds


 ---------- Optimizing ----------
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 5600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 lo

In [6]:
model.saveModel("./saves/alpha_0d5")



 ---------- Saving model and solution ----------
Model saved successfully as './saves/alpha_0d5.mps'
Solution saved successfully as './saves/alpha_0d5.sol'


In [5]:
model = OR_model(df=df, model_path="./saves/alpha_0d5")



 ---------- Loading model from './saves/alpha_0d5.mps'... ----------
Set parameter Username
Set parameter LicenseID to value 2631632
Academic license - for non-commercial use only - expires 2026-03-05
Read MPS format model from file ./saves/alpha_0d5.mps
Reading time = 0.31 seconds
OR: 3654 rows, 48147 columns, 759976 nonzeros
Model loaded successfully!
Read solution from file ./saves/alpha_0d5.sol
Solution loaded successfully from './saves/alpha_0d5.sol'


In [7]:
model.optimize()



 ---------- Optimizing ----------
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 5600X 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3654 rows, 48147 columns and 759976 nonzeros
Model fingerprint: 0x493929bf
Variable types: 0 continuous, 48147 integer (48118 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]

Loaded user MIP start with objective 183.5

Presolve removed 224 rows and 0 columns
Presolve time: 0.70s
Presolved: 3430 rows, 48147 columns, 439803 nonzeros
Variable types: 0 continuous, 48147 integer (48118 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.13s

Solved with dual simplex

Root relaxation: cutoff, 4942 iterations, 1

In [8]:
model.printResults()


Optimal Solution Found!

Decision Variables:
x_p[(d3,o39),(d3,o39)] = 1.0
x_p[(d1,o30),(d1,o30)] = 1.0
x_p[(d3,o8),(d3,o8)] = 1.0
x_p[(d2,o6),(d2,o6)] = 1.0
x_p[(d5,o43),(d5,o43)] = 1.0
x_p[(d7,o16),(d7,o16)] = 1.0
x_p[(d6,o19),(d6,o19)] = 1.0
x_p[(d6,o31),(d6,o31)] = 1.0
x_p[(d1,o20),(d1,o20)] = 1.0
x_p[(d7,o36),(d7,o36)] = 1.0
x_p[(d6,o26),(d6,o26)] = 1.0
x_p[(d1,o38),(d1,o38)] = 1.0
x_p[(d4,o41),(d4,o41)] = 1.0
x_p[(d4,o22),(d4,o22)] = 1.0
x_p[(d3,o32),(d3,o32)] = 1.0
x_p[(d3,o23),(d3,o23)] = 1.0
x_p[(d4,o2),(d4,o2)] = 1.0
x_p[(d5,o33),(d5,o33)] = 1.0
x_p[(d5,o18),(d5,o18)] = 1.0
x_p[(d6,o46),(d6,o46)] = 1.0
x_p[(d7,o30),(d7,o30)] = 1.0
x_p[(d4,o16),(d4,o16)] = 1.0
x_p[(d3,o49),(d3,o49)] = 1.0
x_p[(d6,o39),(d6,o39)] = 1.0
x_p[(d4,o24),(d4,o24)] = 1.0
x_p[(d7,o49),(d7,o49)] = 1.0
x_p[(d6,o22),(d6,o22)] = 1.0
x_p[(d6,o29),(d6,o29)] = 1.0
x_p[(d5,o7),(d5,o7)] = 1.0
x_p[(d5,o2),(d5,o2)] = 1.0
x_p[(d2,o21),(d2,o21)] = 1.0
x_p[(d5,o29),(d5,o29)] = 1.0
x_p[(d1,o52),(d1,o52)] = 1.0
x_p[(d3